In [1]:
import pandas as pd
from flashtext import KeywordProcessor
import os
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import io
import PyPDF2
from PyPDF2 import PdfFileReader
import re
from datetime import datetime
import time
import math
import fitz
import time
from docx import Document
from win32com import client as wc
import docx2txt
from langdetect import detect
from googletrans import Translator

In [2]:
documenttitlef = {'SOW': 'Contract', 'Contract': 'Contract', 'HPE Customer Terms - Portfolio': 'Contract', 'Agreement': 'Contract', 'Addenda': 'Supportive Documents', 'Amendment': 'Supportive Documents', 'Exhibit': 'Supportive Documents', 'Emails': 'Supportive Documents', 'Addendum': 'Supportive Documents', 'PA Request form': 'Supportive Documents', 'Coversheet': 'Supportive Documents', 'Checklist': 'Supportive Documents', 'Customer Quote': 'Supportive Documents', 'Extension': 'Supportive Documents', 'Annexure': 'Supportive Documents', 'Schedule': 'Supportive Documents','Other Documents': 'Supportive Documents'}

In [3]:
def language_criteria():
    sheet_list=['English','Dutch','Spanish','Czech','French','Italian','German','Danish','Portuguese']
    lang_code=['en','nl','es','cs','fr','it','de','da','pt']
    base_dict={name:{} for name in lang_code}

    i=0
    for name in sheet_list:
        df = pd.read_excel("Language Translation.xlsx",sheet_name=name)
        print(name)
        res = []
        keywordsdict = {}
        documenttitledict= {}
        for row in range(0,len(df)):
            word = df['Translation'][row]
            words = word.split(',')
            record = df['Record Type'][row]
            if record not in keywordsdict:
                keywordsdict[record] = []
            for word in words:
                keywordsdict[record].append(word)
#         print(keywordsdict)
        title = df['Document Title'][row]
        documenttitledict[record] = title
    #     base_dict[name]=keywordsdict
        base_dict[lang_code[i]] = keywordsdict
        i+=1
    final_dict={}
    for i in base_dict.keys():
        for j in base_dict[i]:
            if j in final_dict:
                final_dict[j].extend(base_dict[i][j])
            else:
                final_dict.update({j:base_dict[i][j]})
    for key in final_dict.keys():
        re = final_dict[key]
        re = list(set(re))
        final_dict[key] = re
    return final_dict

In [4]:
final_dict = language_criteria()
final_dict

English
Dutch
Spanish
Czech
French
Italian
German
Danish
Portuguese


{'SOW': ['EDT',
  ' Erklæring om arbejde',
  ' Dichiarazione di lavoro',
  ' Statement of Work',
  'SOW',
  ' Declaración de trabajo',
  ' Leistungsbeschreibung',
  ' Declaração de Trabalho',
  ' Verklaring van Werk',
  ' prohlášení o práci',
  ' Énoncé des travaux'],
 'Contract': ['Contrato',
  'Vertrag',
  'Contract',
  'Kontrakt',
  'Contracter',
  'Smlouva',
  'Contrarre'],
 'HPE Customer Terms - Portfolio': ['Termini per i clienti HPE - Portafoglio',
  ' vilkår og betingelser',
  ' términos',
  ' conditions',
  ' Terms',
  'HPE Klantvoorwaarden - Portfolio',
  ' Terms & Conditions',
  'HPE Kundenbedingungen – Portfolio',
  'HPE-kundevilkår - portefølje',
  ' Algemene voorwaarden',
  'Términos del cliente de HPE: cartera',
  ' termos',
  ' termini',
  ' Allgemeine Geschäftsbedingungen',
  ' vilkår',
  ' termini e condizioni',
  'Podmínky zákazníka HPE - portfolio',
  'Termos do cliente HPE - Portfólio',
  ' podmínky',
  'Conditions client HPE - Portefeuille',
  'HPE Customer Terms 

In [5]:
def ocr(pdfpath):
    try:
        pdfFileObj = PyPDF2.PdfFileReader(pdfpath, 'rb')
        a = pdfFileObj.getNumPages()
        doc = fitz.open(pdfpath)
        for i in range(1):
                page = doc.loadPage(i)  # number of page
                pix = page.getPixmap()
                #output = "out_"+str(cnt)+".png"
                output = pdfpath[:-4]+"_"+str(i)+".jpg"
                #print(output)
                pix.writePNG(output)
                im = Image.open(output)
                im.save(output, dpi=(600,600))


        # output = pdfpath[:-4]+"_"+str(1)+".jpg"
        # pix.writePNG(output)
        # im = Image.open(output)
        # im.save(output, dpi=(600,600))


        pytesseract.pytesseract.tesseract_cmd ='C:\\Users\\rnavn\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
        text = str(((pytesseract.image_to_string(Image.open(output)))))
        print(text)
        return text
    except OSError:
        return "fnf err"
    except:
        return "ecr err"

In [6]:
def readdoc(filepath,file_extension):
    try:
        if file_extension == "docx":
            my_text = docx2txt.process(filepath)
            print(my_text)
            return my_text

        #     file_path = r'C:\Users\rnavn\Contact Migration\wer.doc'
        else:
            word = wc.Dispatch("Word.Application")
            doc = word.Documents.Open(filepath)
        #     doc.SaveAs(r'C:\Users\rnavn\Contact Migration\wer2.docx', 12) #12 stands for docx
            filepath = filepath + "x"
            doc.SaveAs(filepath,12)
            doc.Close()
            print(filepath)
            word.Quit()
            my_text = docx2txt.process(filepath)
            print(my_text)
            return my_text
    except OSError:
        return "fnf err"
    except:
        return "ecr err"

In [7]:
def readex(filepath):
    try:
        sp = pd.read_excel(filepath)
        sp = sp.fillna('-')
        # arr = [x for x in range(0,len(sp.columns))]
        # p = pd.DataFrame(sp, columns = arr)
        col = list(sp.columns)
        sp
        finalstr = ""
        sp.columns
        for i in range(len(sp)):
            for j in range(len(col)):
                finalstr += str(sp[col[j]][i]) + " "
        print(finalstr)
        # res = findrecord(finalstr)
        return finalstr
    except OSError:
        return "fnf err"
    except:
        return "ecr err"

In [8]:
def findrecord(textextract):
    sample = textextract
    kp = KeywordProcessor()
    kp.add_keywords_from_dict(final_dict)
    res = kp.extract_keywords(sample)
    staus = ""
    
    priority = ['Extension','Addenda','Amendment','Addendum','Exhibit','Annexure','Coversheet','Checklist','Customer Quote','PA Request form','Contract','Agreement','Schedule','HPE Customer Terms - Portfolio','SOW','Emails','Other Documents']
    # test_list.sort(key = lambda i: sort_order.index(i[0]))
    orderres = list(set(res))
    
    print(orderres)
    if len(orderres) == 0:
        recordtype = ""
        documenttype = ""
        status = "Unavailable"
    else:
        orderres.sort(key = lambda i: priority.index(i))
        print(orderres)
        recordtype = orderres[0]
        documenttype = documenttitlef[orderres[0]]
        status = "Found"
#     res = []
#     res.append([filename,recordtype,documenttype])
#     if len(orderres) >= 2:
#         recordtype = recordtype + "More than one"
    ans = [recordtype,documenttype,status]
    return ans

In [9]:
def readdata(finalpath,file_extension):
    docs = set(['doc','docx','DOC','DOCX'])
    pdfs = set(['pdf','PDF'])
    exc = set(['xlsx','xls','XLS','XLSX'])
    other = set(['zip','ZIP','xlsb','xlsm','XLSM','XLSB'])
    if file_extension in pdfs:
        textextract = ocr(finalpath)
    elif file_extension in docs:
        textextract = readdoc(finalpath,file_extension)
    elif file_extension in exc:
        textextract = readex(finalpath)
    else:
        textextract = "None"
    return textextract

In [10]:
ft = pd.read_excel('manualemea.xlsx')
ft

,Column1,PA No,File Name,Path,Customer Name,Record Type,Document Title,PA No Availability Status,Record Type Availability Status,File extension,COAP PA no,Future check reference - Manual
0,5445,NaN,ACCOR.pdf,/EMEAContracts/Contracts/ACCOR,ACCOR,NaN,NaN,Active,NaN,pdf,FG1AC,Manual
1,7780,NaN,Renewal Letter Jan02.doc,/EMEAContracts/Contracts/Eurocontrol,Eurocontrol,NaN,NaN,Active,NaN,doc,BM4EC,Manual
2,7791,NaN,PA-INFO_BG147.doc,/EMEAContracts/Contracts/Eurocontrol/005 - JAN...,Eurocontrol,NaN,NaN,Active,NaN,doc,BM4EC,Manual
3,14235,NaN,ALFA LAVAL.pdf,/EMEAContracts/Contracts/Alfa Laval - GPOE1200,ALFA LAVAL,NaN,NaN,Active,NaN,pdf,SM1AL,Manual
4,14812,NaN,AUTOLIV AB.pdf,/EMEAContracts/Contracts/Autoliv,AUTOLIV,NaN,NaN,Active,NaN,pdf,A251I,Manual
5,14948,NaN,Tietokesko-Termination-Letter.doc,/EMEAContracts/Contracts/Tietokesko,Tieto,NaN,NaN,Active,NaN,doc,LG1TI,Manual
6,14989,NaN,AVIS.pdf,/EMEAContracts/Contracts/AVIS/AVIS,AVIS,NaN,NaN,Active,NaN,pdf,BM3AV,Manual
7,15123,NaN,BarclaysBankE16.pdf,/EMEAContracts/Contracts/Barclays Bank (NonStop),Barclays,NaN,NaN,Active,NaN,pdf,UM4YQ,Manual
8,15318,NaN,Wincor Nixdorf system aggregator.doc,/EMEAContracts/Contracts/Wincor-Nixdorf,Wincor Nixdorf,NaN,NaN,Active,NaN,doc,UM4ZW,Manual
9,15319,NaN,Wincor Nixdorf system aggregatorD24D1.doc,/EMEAContracts/Contracts/Wincor-Nixdorf,Wincor Nixdorf,NaN,NaN,Active,NaN,doc,UM4ZW,Manual


In [12]:
new = 0
ocrsucess = 0
noteng = 0
errocr = 0
result = []
fnfocr = 0
ecrocr = 0
bt = set([-1,26402])
for i in range(0,len(ft)):
    print(i)
    file_paths=[]
    pa_no = ft['PA No'][i]
    file_name = ft['File Name'][i]
    path = ft['Path'][i]
    customer_name = ft['Customer Name'][i]
    record_type = ft['Record Type'][i]
    document_title = ft['Document Title'][i]
    pa_availability = ft['PA No Availability Status'][i]
    record_availability = ft['Record Type Availability Status'][i]
    file_extension = ft['File extension'][i]
    ex = set(['pdf','PDF','doc','DOC','docx','DOCX','xls','XLS','xlsx','XLSX','XLSM','xlsm','XLSB','xlsb','ZIP','zip'])
    #print(file_name)
    if pd.isnull(record_availability) and pa_availability == "Active" and i not in bt:
        if file_extension not in ex:
            record_availibility = "Available"
            record_type = "Other Documents"
            document_title = "Supporting Documents"
#             continue
        else:
            filepath = path.replace("/","\\")
            ap = "C:\\Users\\rnavn\\OneDrive - Hewlett Packard Enterprise"
            finalpath = ap + filepath + "\\" + file_name
            #print(finalpath)
            #print(type(filepath))
    #         file_paths.append(filepath)
    #         if filepath in folder_list:
                #print("HERE")
                #print("FINALPATH",finalpath)
            textextract = readdata(finalpath,file_extension)
                #print(textextract)
            if textextract == "None":
                    new += 1
            elif textextract == "fnf err":
                    record_availability = "File Not Found Error"
                    fnfocr += 1
            elif textextract == "ecr err":
                    record_availability = "Encryption Error"        

            else:
    #                 keywordsdict= language_criteria()
#                     translator = Translator()
#                     t = translator.translate(textextract)
#                     transtext = t.text
#                     finalans = findrecord(transtext)
                    finalans = findrecord(textextract)
                    record_availability = finalans[2]
                    record_type = finalans[0]
                    document_title = finalans[1]
                    ocrsucess += 1
                   
#     print([pa_no,file_name,path,customer_name,record_type,document_title,pa_availability,record_availability,file_extension])
    result.append([pa_no,file_name,path,customer_name,record_type,document_title,pa_availability,record_availability,file_extension])


0
Bee: P.LELINTHE
-

 

psa compur hay onan Oeste

ACCOR

Mr Masson Fabrice
‘2rue de la mare neuve
91021 Evry cedex
France

20" July 1999
Extension of Atlas Agreement
Dear Sir,
‘Since the Atlas Agreement between ACCOR and Compag computer will expire on 31° July
1999, we would like to confirm to you that we will extend the duration ofthe agreement by
12 months until 1 August 2000,

During this extension period all conditions and procedures outlined in the above mentioned
‘agreement will continue to apply as practiced during the past year.

‘We would like to take this opportunity to thank you for your trust in our products and look
forward to a continued successful business relationship.

Yours faithfully,
‘Compaq Computer EMEA BV

for Patrick Starck

Vice-president, Global Accounts EMEA

COMPA, errors tompaconpa itis evens Contre Conmera Heese

['Extension', 'Agreement']
['Extension', 'Agreement']
1
C:\Users\rnavn\OneDrive - Hewlett Packard Enterprise\EMEAContracts\Contracts\Eurocontro

['Other Documents', 'Contract', 'Exhibit', 'Agreement', 'HPE Customer Terms - Portfolio', 'Schedule']
['Exhibit', 'Contract', 'Agreement', 'Schedule', 'HPE Customer Terms - Portfolio', 'Other Documents']
9
C:\Users\rnavn\OneDrive - Hewlett Packard Enterprise\EMEAContracts\Contracts\Wincor-Nixdorf\Wincor Nixdorf system aggregatorD24D1.docx
HP ELECTRONIC SOLUTION CONFIDENTIAL DISCLOSURE AGREE	Exhibit CDA



INTERNET EXHIBIT	Exhibit IE



PROJECT PROVISIONS	Exhibit PROJ







SYSTEM AGGREGATOR HP SURESTORE E DISK ARRAY XP256 PRODUCTS	Exhibit B1235

RESELLERS OF WINCOR	Exhibit WINRES







COMPUTER SYSTEM AGGREGATOR AGREEMENT	Exhibit CSA





SA ADDED VALUE AND DISCOUNT CLAUSE	Exhibit RSAP





DEVELOPMENT, TESTING & DEMO EXHIBIT	Exhibit E104





SYSTEM AGGREGATOR COMPUTER SYSTEM PRODUCTS	Exhibit A3001





SYSTEM AGGREGATOR HP ENTRY-LEVEL ENTERPRISE COMPUTER PRODUCTS	Exhibit B1198





HP OPENVIEW NT-CENTRIC  SOLUTIONS	Exhibit AV26





HP OPENVIEW E MGMT & E SERVICES SOFTWARE	Exhibit 

['Other Documents', 'Contract', 'Exhibit', 'Agreement', 'HPE Customer Terms - Portfolio', 'Schedule']
['Exhibit', 'Contract', 'Agreement', 'Schedule', 'HPE Customer Terms - Portfolio', 'Other Documents']
10
BACKARO

   

 

General Terms Agreement

Dispositions et conditions générales convenues entre BELGACOM S.A. de Droit Public et

 

Configurations informatiques passées par BELGACOM - General Services Division -IS-
Purchases et renvoyant expicitement au présent GTA.
Dénomination du présent GTA:

1998.

 

 

 

‘ume part,

BELGACOM, socité anonyme do rot public
Boulevard E. Jacgmain 177 ia

31030 Gruxoles

TVA BE 202.239.951

RC. Snxcles $87 163

représentée par Nonsieur G. WANET, GES Division Gonerat Manager, et Monsieur D. KETELS,
Drector Purchasing,

haprés dénommée « BELGACOM »,

Et'auve pan,

HEWLETT-PACKARD BELGIUM SA

Boulevard de a Woluwe 160

15-1200 Bruxoles

TVA BE 402220.504

RG. sidge socal

RC. ldge despltaion i
‘eprésentée par Monsieur M. ROTTHIER, Managing Directer.


HP Direktvertrag
Vertrags-Nr. DM2AU

wsehan wa
Hove Paar Gb ‘NUDIAG.
errenberger Sabie 140,

71034 Botingen 2045 ingot
He wa)

\Verregibagn 01, Mal 2008

1, nant es Vereages,Gatigh

leer Varrag hat ine Laut von zu 12 Monate. EF veringet sch autmatsch un Jewels woe 12 Mena,
Sefer or ict ve sine der beiden Fain une Enhaung cer Fst von orig (30) Tagen 2um Ende des Jowellgen

 

 

[]
15
Rahmenvertrag
(Master Contract)

ber Leistungen auf dem Gebiet der
Informationstechnologie (IT)
HP Vertragsnummer: DG62B
= Beschaffung von
Desktops, Notebooks, Workstations, Bildschirmen,

‘Druckem, Multfunktionsgeraten, Scannem,
‘Server-, Storage- und Netzwerkprodukten

mit entsprechenden Zusatzkomponenten und Zubehor -

(nachfolgend ,Rahmenvertrag™ genannt)
‘awischen der

Volkswagen Aktiengeselischaft
Berliner Ring 2
38436 Wolfsburg

(nachfolgend Auftraggeber oder AG" genannt)
und der

Hewlett-Packard GmbH
Herrenberger Strate 140
71034 Boblingen

(nachfolgend Auftragnehmer’ oder AN" genannt)

(AG und AN

______ Heineken

 

Nederland Business Services

 

 

 

5 (evo peer Ske
Remonnseer ty Postbus 9064
308 08 Uvesh
PSWGE 2008041 O76 6806 26 augustus 2004

Side letter behorende bij overeenkomst BST 200804

+ Penalties and Claims

(a neat 10 vn Aachen C: Serve Level Requiem van et etc

Partjen gaan akkoord om uit to gaan van do standaard inkoopvoowaarden van Heineken
‘Nederland 8.V. (ze Attachment A van het contact) eno het Nederlands recht

 

+ Prien

attien komen overeen dat genoemde przen (zie Altachment E van het contact) gonerick golden
‘oor leveringen binnen Nederland

+ Asset tagging

LHP biedt Heineken aan om asset-tagging ult to voeren voor rekening van HP indien HP dit conform
liens standaard format en dens standaard procedures mag uitvoeren,

+ Levertja

‘Ze onderstaand DOA / Levert / Garantie,

‘+ DOA Lovertid/ Garantie

‘De levertj¢ van 8 werkdagen zal gehandhaatd bijven, echter om aan deze els te voldoen zal HP om
‘et een reservevoorraad van 5 notebooks en 5 desktops aan 

In [13]:
r = pd.DataFrame(result,columns = ['PA No','File Name','Path','Customer Name','Record Type','Document Title','PA No Availability Status','Record Type Availability Status','File extension'])
r.style


,PA No,File Name,Path,Customer Name,Record Type,Document Title,PA No Availability Status,Record Type Availability Status,File extension
0,nan,ACCOR.pdf,/EMEAContracts/Contracts/ACCOR,ACCOR,Extension,Supportive Documents,Active,Found,pdf
1,nan,Renewal Letter Jan02.doc,/EMEAContracts/Contracts/Eurocontrol,Eurocontrol,Contract,Contract,Active,Found,doc
2,nan,PA-INFO_BG147.doc,/EMEAContracts/Contracts/Eurocontrol/005 - JAN-DEC2004,Eurocontrol,Exhibit,Supportive Documents,Active,Found,doc
3,nan,ALFA LAVAL.pdf,/EMEAContracts/Contracts/Alfa Laval - GPOE1200,ALFA LAVAL,Contract,Contract,Active,Found,pdf
4,nan,AUTOLIV AB.pdf,/EMEAContracts/Contracts/Autoliv,AUTOLIV,Agreement,Contract,Active,Found,pdf
5,nan,Tietokesko-Termination-Letter.doc,/EMEAContracts/Contracts/Tietokesko,Tieto,Contract,Contract,Active,Found,doc
6,nan,AVIS.pdf,/EMEAContracts/Contracts/AVIS/AVIS,AVIS,Agreement,Contract,Active,Found,pdf
7,nan,BarclaysBankE16.pdf,/EMEAContracts/Contracts/Barclays Bank (NonStop),Barclays,Agreement,Contract,Active,Found,pdf
8,nan,Wincor Nixdorf system aggregator.doc,/EMEAContracts/Contracts/Wincor-Nixdorf,Wincor Nixdorf,Exhibit,Supportive Documents,Active,Found,doc
9,nan,Wincor Nixdorf system aggregatorD24D1.doc,/EMEAContracts/Contracts/Wincor-Nixdorf,Wincor Nixdorf,Exhibit,Supportive Documents,Active,Found,doc


In [14]:
r.to_excel('emea_manual_report.xlsx')

In [32]:
print(fnfocr,ecrocr,ocrsucess)

0 0 72


In [ ]:
# 'greeting' - [hey,heelo,hi]
# 'good bye' - [bye, see ya]
# text = Hey there whats up, see ya
#  ['greeting','good bye']